In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-21"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12388,2024-04-21,Brasil Nbb,16:30,Pato,Bauru,1.79,1.93,159.5,1.87,1.79,-1.5,1.92,2.10,KSFtj4YQ,0.558659,0.518135,0.534759,0.558659,0.076794,0.6,0.6,NaN,NaN,242.572,171.920693,0.708741,131.750,16.191638,0.122897,219.538,181.400,149.48,148.20,0.0,0.0,0.0,0.0,0.053223,0.030912,0.063323,2.03,0.406,1.945813,0.490020,0.5,0.009980,4.76,0.952,0.976891,0.470610,0.5,0.029390
12389,2024-04-21,Brasil Nbb,18:30,São José,Fortaleza B.C.,1.49,2.50,155.5,1.88,1.79,-4.5,1.87,3.75,IqZ3gwuJ,0.671141,0.400000,0.531915,0.558659,0.071141,0.4,0.6,NaN,NaN,138.376,38.520056,0.278372,131.154,19.657581,0.149882,116.180,180.006,124.16,141.96,0.0,0.0,0.0,0.0,0.357984,0.034681,0.473082,-0.77,-0.154,-3.181818,0.669480,0.6,-0.069480,1.33,0.266,5.639098,0.398510,0.3,-0.098510
12390,2024-04-21,Espanha Acb,13:30,Unicaja,Baskonia,1.28,3.75,160.5,1.84,1.96,-11.5,1.89,9.50,MkNEUzgb,0.781250,0.266667,0.543478,0.510204,0.047917,0.4,0.8,NaN,NaN,144.324,34.811432,0.241203,176.888,59.662482,0.337290,137.996,168.982,185.49,242.40,0.0,0.0,0.0,0.0,0.694455,0.044659,0.944878,0.68,0.136,2.058824,0.000000,0.0,0.000000,-1.94,-0.388,-7.087629,0.000000,0.0,0.000000
12391,2024-04-21,Eua Nba,16:30,Los Angeles Clippers,Dallas Mavericks,2.27,1.68,222.5,1.80,1.95,1.5,2.05,1.53,lKosmMFr,0.440529,0.595238,0.555556,0.512821,0.035767,0.8,0.2,NaN,NaN,226.306,37.836995,0.167194,207.036,44.002415,0.212535,216.608,216.216,253.05,172.05,0.0,0.0,0.0,0.0,0.211237,0.056569,0.205416,-3.48,-0.696,-1.824713,0.655006,0.4,-0.255006,3.96,0.792,0.858586,0.557893,0.8,0.242107
12392,2024-04-21,Eua Nba,20:00,Milwaukee Bucks,Indiana Pacers,2.02,1.85,231.5,1.93,1.93,1.5,1.89,1.63,MXqko0pe,0.495050,0.540541,0.518135,0.518135,0.035590,0.6,0.4,NaN,NaN,196.510,37.060807,0.188595,202.368,37.039478,0.183030,209.230,245.892,236.34,260.40,0.0,0.0,0.0,0.0,0.062123,0.000000,0.104459,3.84,0.768,1.328125,0.612326,0.7,0.087674,-3.17,-0.634,-1.340694,0.541861,0.3,-0.241861
12393,2024-04-21,Itália Liga A,13:15,Virtus Bologna,Reggiana,1.19,4.79,159.5,1.86,1.94,-9.5,1.82,10.50,AkIByMbS,0.840336,0.208768,0.537634,0.515464,0.049104,0.6,0.2,NaN,NaN,132.744,21.742573,0.163793,182.140,71.655449,0.393409,114.700,177.180,136.92,161.54,0.0,0.0,0.0,0.0,0.851366,0.029773,0.996378,1.20,0.240,0.791667,0.837608,0.9,0.062392,-3.22,-0.644,-5.885093,0.453260,0.2,-0.253260
12394,2024-04-21,Alemanha Bbl,12:00,Gottingen,Chemnitz,4.39,1.22,168.5,1.82,1.96,10.5,1.87,1.04,h6Tled4r,0.227790,0.819672,0.549451,0.510204,0.047463,0.6,0.8,NaN,NaN,313.726,284.741654,0.907613,163.382,94.883489,0.580746,229.486,204.060,135.75,332.00,0.0,0.0,0.0,0.0,0.799119,0.052378,0.403367,0.13,0.026,130.384615,0.442626,0.4,-0.042626,-0.44,-0.088,-2.500000,0.000000,0.0,0.000000
12395,2024-04-21,Alemanha Pro B,13:00,Frankfurt 2,Wedel,1.74,1.99,152.5,1.80,1.88,-1.5,1.85,2.08,vLA7kQlI,0.574713,0.502513,0.555556,0.531915,0.077225,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,148.23,0.00,0.0,0.0,0.0,0.0,0.094786,0.030744,0.082766,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12396,2024-04-21,Argentina Liga A,21:00,Regatas,Instituto de Cordoba,1.94,1.78,155.5,1.85,1.85,1.5,1.81,1.64,ht0VDGrP,0.515464,0.561798,0.540541,0.540541,0.077262,0.4,0.6,NaN,NaN,141.660,20.551797,0.145078,131.260,26.988302,0.205609,142.768,141.670,117.81,158.10,0.0,0.0,0.0,0.0,0.060826,0.000000,0.069686,0.65,0.130,7.230769,0.000000,0.0,0.000000,-1.61,-0.322,-2.422360,0.653415,0.7,0.046585
12397,20

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,16:30,Brasil Nbb,Pato,Bauru,159.5,1.87,1.0000,Over
1,18:30,Brasil Nbb,São José,Fortaleza B.C.,155.5,1.88,1.0000,Over
2,13:30,Espanha Acb,Unicaja,Baskonia,160.5,1.84,1.0000,Over
3,13:15,Itália Liga A,Virtus Bologna,Reggiana,159.5,1.86,1.0000,Over
4,11:00,Europa Liga Bnxt,LWD Basket,Kortrijk,166.5,1.80,0.9904,Over
5,14:05,França Lnb,Lyon-Villeurbanne,Paris,164.5,1.87,1.0000,Over
6,10:00,Grécia Liga De Elite,Aias Evosmou,Panerythraikos,148.5,1.80,1.0000,Over
7,09:30,Irã Superliga,Tabiat,Palayesh Naft Abadan,163.5,1.85,1.0000,Over
8,14:30,Israel Superliga,Hapoel Holon,Hapoel Jerusalem,156.5,1.86,1.0000,Over
9,13:00,Itália Série A2,Agrigento,Orzinuovi,157.5,1.87,1.0000,Over
